In [13]:
%cd ABIDE-FMRI-ConnectivityMatrix_ML_DL/notebooks

[Errno 2] No such file or directory: 'ABIDE-FMRI-ConnectivityMatrix_ML_DL/notebooks'
/Users/sabrinebendimerad/Downloads/ABIDE-FMRI-ConnectivityMatrix_ML_DL


In [15]:
import numpy as np
import h5py
import pandas as pd
import utils
from utils import (load_phenotypes, format_config, run_progress, hdf5_handler,load_fold)
from sklearn.model_selection import StratifiedKFold, train_test_split

In [16]:
pheno_path = "./data/phenotypes/Phenotypic_V1_0b_preprocessed1.csv"
pheno = load_phenotypes(pheno_path)

In [17]:
pheno.head()

,FILE_ID,DX_GROUP,SEX,SITE_ID,MEAN_FD,SUB_IN_SMP,STRAT,AGE
FILE_ID,,,,,,,,
Pitt_0050003,Pitt_0050003,0,1,PITT,0.322092,True,PITT_0,24.45
Pitt_0050004,Pitt_0050004,0,1,PITT,0.127745,True,PITT_0,19.09
Pitt_0050005,Pitt_0050005,0,2,PITT,0.128136,False,PITT_0,13.73
Pitt_0050006,Pitt_0050006,0,1,PITT,0.070143,True,PITT_0,13.37
Pitt_0050007,Pitt_0050007,0,1,PITT,0.151246,True,PITT_0,17.78


In [18]:
# Open the HDF5 file in read mode
file_path = "./data/abide.hdf5"
hdf5_file = h5py.File(file_path, "r")


In [19]:
# This will give us a quick overview of the main groups in the HDF5 file.
def list_top_level_items(hdf5_file):
    for key in hdf5_file.keys():
        print(f"Top-level item: {key}")

list_top_level_items(hdf5_file)


Top-level item: experiments
Top-level item: patients


In [20]:
# This cell will print all the items within the 'patients' group.
def explore_group(group_name):
    if group_name in hdf5_file:
        group = hdf5_file[group_name]
        print(f"Exploring '{group_name}' group:")
        for key in group.keys():
            print(f"  {key}: {type(group[key])}")
    else:
        print(f"'{group_name}' group not found in the HDF5 file.")

explore_group("patients")


Exploring 'patients' group:
  CMU_a_0050642: <class 'h5py._hl.group.Group'>
  CMU_a_0050646: <class 'h5py._hl.group.Group'>
  CMU_a_0050647: <class 'h5py._hl.group.Group'>
  CMU_a_0050649: <class 'h5py._hl.group.Group'>
  CMU_a_0050653: <class 'h5py._hl.group.Group'>
  CMU_a_0050654: <class 'h5py._hl.group.Group'>
  CMU_a_0050656: <class 'h5py._hl.group.Group'>
  CMU_a_0050659: <class 'h5py._hl.group.Group'>
  CMU_a_0050660: <class 'h5py._hl.group.Group'>
  CMU_a_0050663: <class 'h5py._hl.group.Group'>
  CMU_a_0050664: <class 'h5py._hl.group.Group'>
  CMU_a_0050665: <class 'h5py._hl.group.Group'>
  CMU_a_0050666: <class 'h5py._hl.group.Group'>
  CMU_a_0050668: <class 'h5py._hl.group.Group'>
  CMU_b_0050643: <class 'h5py._hl.group.Group'>
  CMU_b_0050644: <class 'h5py._hl.group.Group'>
  CMU_b_0050645: <class 'h5py._hl.group.Group'>
  CMU_b_0050648: <class 'h5py._hl.group.Group'>
  CMU_b_0050650: <class 'h5py._hl.group.Group'>
  CMU_b_0050651: <class 'h5py._hl.group.Group'>
  CMU_b_0050

In [21]:
#Inspect a sample patient record (replace 'PATIENT_ID' with an actual ID you find in the 'patients' group)
sample_patient_id = "Yale_0050554"  # Replace with an actual ID from the previous output

if sample_patient_id in hdf5_file["patients"]:
    sample_patient = hdf5_file["patients"][sample_patient_id]
    print(f"Attributes for patient '{sample_patient_id}':")
    for attr_name, attr_value in sample_patient.attrs.items():
        print(f"  {attr_name}: {attr_value}")

    print("\nDatasets for patient:")
    for dataset_name in sample_patient.keys():
        print(f"  {dataset_name}: {sample_patient[dataset_name].shape}")
else:
    print(f"Patient '{sample_patient_id}' not found.")


Attributes for patient 'Yale_0050554':
  id: Yale_0050554
  sex: 1
  site: YALE
  y: 1

Datasets for patient:
  aal: (6670,)
  cc200: (19900,)
  dosenbach160: (12880,)
  ez: (6670,)
  ho: (6105,)
  tt: (4656,)


In [22]:
#Explore the 'experiments' group (if it exists)
explore_group("experiments")

Exploring 'experiments' group:
  aal_whole: <class 'h5py._hl.group.Group'>
  cc200_whole: <class 'h5py._hl.group.Group'>
  dosenbach160_whole: <class 'h5py._hl.group.Group'>
  ez_whole: <class 'h5py._hl.group.Group'>
  ho_whole: <class 'h5py._hl.group.Group'>
  tt_whole: <class 'h5py._hl.group.Group'>


In [23]:
#Inspect a specific experiment (replace 'EXPERIMENT_NAME' with an actual experiment you find)
experiment_name = "cc200_whole"  # Replace with an actual experiment name
fold_id = "1"  # Usually, folds are numbered, so let's inspect the first one

if experiment_name in hdf5_file["experiments"]:
    experiment_group = hdf5_file["experiments"][experiment_name]
    if fold_id in experiment_group:
        fold_group = experiment_group[fold_id]
        print(f"Fold '{fold_id}' contents in '{experiment_name}':")
        for key in fold_group.keys():
            print(f"  {key}: {len(fold_group[key])} items")
    else:
        print(f"Fold '{fold_id}' not found in experiment '{experiment_name}'.")
else:
    print(f"Experiment '{experiment_name}' not found.")


Fold '1' contents in 'cc200_whole':
  test: 104 items
  train: 623 items
  valid: 308 items


In [24]:
# Open the HDF5 file in read-only mode to prevent accidental overwriting
with h5py.File(file_path, 'r') as hdf5_file:
    # Step 1: List all experiments in the "experiments" group
    experiments = list(hdf5_file["experiments"].keys())
    print("Available experiments:", experiments)

    # Step 2: Choose a specific experiment to inspect
    experiment_name = experiments[0]  # For example, choose the first experiment
    experiment_group = hdf5_file["experiments"][experiment_name]
    print(f"\nInspecting Experiment: {experiment_name}")

    # Step 3: List all folds within the experiment
    folds = list(experiment_group.keys())
    print("Available folds in the experiment:", folds)

    # Step 4: Choose a specific fold to inspect (e.g., the first fold)
    fold_name = folds[0]
    fold_group = experiment_group[fold_name]
    print(f"\nInspecting Fold: {fold_name}")

    # Step 5: Explore the contents of the train, valid, and test datasets in this fold
    for data_type in ["train", "valid", "test"]:
        data = fold_group[data_type][:]
        print(f"\n{data_type.capitalize()} Data:")
        print(data)


Available experiments: ['aal_whole', 'cc200_whole', 'dosenbach160_whole', 'ez_whole', 'ho_whole', 'tt_whole']

Inspecting Experiment: aal_whole
Available folds in the experiment: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

Inspecting Fold: 0

Train Data:
[b'UCLA_1_0051214' b'Caltech_0051458' b'UCLA_1_0051237' b'USM_0050490'
 b'USM_0050525' b'MaxMun_a_0051606' b'NYU_0051033' b'USM_0050468'
 b'Pitt_0050022' b'KKI_0050784' b'NYU_0051105' b'Trinity_0050255'
 b'Trinity_0051136' b'UM_2_0050426' b'UM_2_0050424' b'Caltech_0051484'
 b'UM_1_0050299' b'USM_0050530' b'MaxMun_c_0051344' b'OHSU_0050149'
 b'CMU_a_0050665' b'Stanford_0051184' b'KKI_0050794' b'Leuven_1_0050687'
 b'SDSU_0050197' b'Leuven_2_0050739' b'Leuven_1_0050710' b'UCLA_2_0051294'
 b'Trinity_0050264' b'Leuven_2_0050730' b'SDSU_0050202' b'Yale_0050563'
 b'Yale_0050564' b'USM_0050519' b'NYU_0050998' b'Pitt_0050049'
 b'Leuven_2_0050755' b'UCLA_1_0051273' b'Caltech_0051481' b'KKI_0050793'
 b'Leuven_2_0050736' b'USM_0050469' b'C

In [26]:
# Path to the HDF5 file and phenotypic data file
hdf5_path = "./data/abide.hdf5"

# Step 1: Load phenotypic data
pheno = load_phenotypes(pheno_path)
print("Phenotypic Data Sample:")
print(pheno.head())

# Step 2: Open HDF5 file
with h5py.File(hdf5_path, 'r') as hdf5:
    # List experiments
    print("\nAvailable experiments in HDF5:", list(hdf5["experiments"].keys()))

    # Choose an experiment and a fold to inspect
    experiment = list(hdf5["experiments"].keys())[0]  # First experiment
    fold = list(hdf5["experiments"][experiment].keys())[0]  # First fold
    print(f"\nExploring Experiment: {experiment}, Fold: {fold}")

    # Step 3: Load fold data
    X_train, y_train, X_valid, y_valid, X_test, y_test = load_fold(hdf5["patients"], hdf5["experiments"][experiment], fold)

    # Step 4: Inspect X_train and y_train
    print("\nX_train Sample:")
    print(X_train[:2])  # Display the first 2 entries in X_train to understand structure
    print(X_train.shape)
    print(X_valid.shape)
    print(X_test.shape)



    print("\ny_train Sample:")
    print(y_train[:10])  # Display the first 10 entries in y_train to understand structure

# This should give insights into what is stored in X_train and y_train, and verify if phenotypic data is included.


Phenotypic Data Sample:
                   FILE_ID  DX_GROUP  SEX SITE_ID   MEAN_FD  SUB_IN_SMP  \
FILE_ID                                                                   
Pitt_0050003  Pitt_0050003         0    1    PITT  0.322092        True   
Pitt_0050004  Pitt_0050004         0    1    PITT  0.127745        True   
Pitt_0050005  Pitt_0050005         0    2    PITT  0.128136       False   
Pitt_0050006  Pitt_0050006         0    1    PITT  0.070143        True   
Pitt_0050007  Pitt_0050007         0    1    PITT  0.151246        True   

               STRAT    AGE  
FILE_ID                      
Pitt_0050003  PITT_0  24.45  
Pitt_0050004  PITT_0  19.09  
Pitt_0050005  PITT_0  13.73  
Pitt_0050006  PITT_0  13.37  
Pitt_0050007  PITT_0  17.78  

Available experiments in HDF5: ['aal_whole', 'cc200_whole', 'dosenbach160_whole', 'ez_whole', 'ho_whole', 'tt_whole']

Exploring Experiment: aal_whole, Fold: 0

X_train Sample:
[[ 0.50574046  0.28057984  0.1369714  ...  0.681499    1.
  10